<a href="https://colab.research.google.com/github/FerdinandPyCode/ArtificialIntelligence/blob/main/fairseq_model_fon_fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

sample_data


In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/test.fon           
  inflating: data/test.fr            
  inflating: data/train.fon          
  inflating: data/train.fr           
  inflating: data/validation.fon     
  inflating: data/validation.fr      


In [3]:
!ls

data  data.zip	sample_data


In [4]:
!git clone https://github.com/pytorch/fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 34933, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 34933 (delta 57), reused 54 (delta 35), pack-reused 34850
Receiving objects: 100% (34933/34933), 25.08 MiB | 21.99 MiB/s, done.
Resolving deltas: 100% (25347/25347), done.


In [5]:
!ls fairseq/

CODE_OF_CONDUCT.md  fairseq	   LICENSE	   RELEASE.md	     setup.py
CONTRIBUTING.md     fairseq_cli    MANIFEST.in	   release_utils.py  tests
docs		    hubconf.py	   pyproject.toml  scripts	     train.py
examples	    hydra_plugins  README.md	   setup.cfg


In [ ]:
!pip install --editable ./fairseq/.

In [ ]:
# Preprocess/binarize the data

!fairseq-preprocess --source-lang fon --target-lang fr \
    --trainpref data/train --validpref data/validation --testpref data/test \
    --destdir data-bin/fonfr.tokenized.fon-fr \
    --workers 20

In [ ]:
!pip install fastBPE sacremoses subword_nmt

In [ ]:
!CUDA_VISIBLE_DEVICES=0
!fairseq-train \
    data-bin/fonfr.tokenized.fon-fr \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --no-epoch-checkpoints \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

In [9]:
!ls

checkpoint_best.pt  data  data-bin  data.zip  fairseq  sample_data


In [10]:
!mkdir checkpoints

mv: cannot move 'checkpoint_best.pt' to 'checkpoints/': Not a directory


In [11]:
!mv checkpoint_best.pt checkpoints/

In [12]:
!fairseq-generate data-bin/fonfr.tokenized.fon-fr \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
D-5434	-0.04362211003899574	elle m’a ajouté deux bananes par dessus le marché
P-5434	-0.0806 -0.0625 -0.0245 -0.0424 -0.0466 -0.0553 -0.0415 -0.0397 -0.0106 -0.0324
S-5401	bɔn ye mɛ ko nɔ blo mɔ lɛ wɛ
T-5401	bon ce sont des gens qui déjà font ainsi
H-5401	-0.043470967561006546	bon ce sont des gens qui déjà font ainsi
D-5401	-0.043470967561006546	bon ce sont des gens qui déjà font ainsi
P-5401	-0.0656 -0.0723 -0.0476 -0.0475 -0.0399 -0.0401 -0.0039 -0.0121 -0.0756 -0.0302
S-5019	zun kpɛn zun tɔn wan ka kpɛn wan tɔn
T-5019	le marteau a son poids <<unk>> a le sien
H-5019	-1.822428584098816	il est lourd à sa place
D-5019	-1.822428584098816	il est lourd à sa place
P-5019	-1.4389 -0.0545 -3.5169 -4.4794 -0.2883 -2.9488 -0.0301
S-4875	un kan byɔ ε ɖɔ tɔ tɔn ku aji
T-4875	je lui ai demandé si son père est mort
H-4875	-0.04703078791499138	je lui ai demandé si son père est mort
D-4875	-0.04703078791499138	je lui ai dem

In [ ]:
!pip install tensorboardX

In [37]:
!echo "Nɛ a ɖe gbɔn ?" | fairseq-interactive data-bin/fonfr.tokenized.fon-fr \
    --path checkpoints/checkpoint_best.pt \
    --beam 5 --source-lang fon --target-lang fr

2023-11-17 01:03:30.096072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 01:03:30.096125: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 01:03:30.096151: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 01:03:30.103928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 01:03:31.096952: W tensorflow/compiler/

In [38]:
!echo "ahan syɛnsyɛn tɛ un ɖo na xwle we" | fairseq-interactive data-bin/fonfr.tokenized.fon-fr \
    --path checkpoints/checkpoint_best.pt \
    --beam 5 --source-lang fon --target-lang fr

2023-11-17 01:05:07.745354: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 01:05:07.745403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 01:05:07.745444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 01:05:07.753424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 01:05:08.756880: W tensorflow/compiler/